In [434]:
#!/usr/bin/env python
# coding: utf-8

#Bibliotecas

import os, email, nltk, numpy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm

def tokenize_email(fullpath):
    
    # esta função recebe um caminho de arquivo de email e extrai um vetor de palavras
    # ao extrair o vetor de palavras, estas recebem alguns tratamentos, descritos a seguir
    
    try:
        # abre o arquivo de email e lê os dados do arquivo
        open_email = open(fullpath, "r")
        read_email = open_email.read()
        # extrai a mensagem do email, e depois o corpo do email
        data_email = email.message_from_string(read_email) 
        message_email = data_email.get_payload() 
        # cria um reconhecedor de padrões e os extrai, criando um vetor de elementos reconhecidos
        tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
        tokens_email = tokenizer.tokenize(message_email)
        # coloca todas as letras em minúsculas
        tolower_email = [word.lower() for word in tokens_email]
        # substitui todos os números encontrados pela string 'numeros'
        for i, token in enumerate(tolower_email):
            if token.isnumeric():
                tolower_email[i] = 'numeros'
        # remove as 'stopwords'
        stopwords = nltk.corpus.stopwords.words('english')
        stopwords.append('_______________________________________________')
        stopped_email = [word for word in tolower_email if word not in stopwords]
        # retorna o email tokenizado
        return(stopped_email)
        
    except:
        # caso ocorra algum erro, retorna FALSO
        return(0)

def tokenize_directory(directory):
    
    # essa função aplica a função tokenize_email a todos os arquivos de um diretório dado

    # inicializando a lista de emails tokenizados
    lista = []

    #iterando sobre os arquivos do diretório
    for file in os.listdir(directory):
    
        # extraindo e passando o parâmetro para a função tokenize_email
        filename = os.fsdecode(file)
        fullpath = os.path.join(directory, filename)
        email_tokens = tokenize_email(fullpath)
        
        # checa se a tokenização obteve sucesso, e no caso positivo adiciona o email tokenizado à lista
        if email_tokens:
            lista.append(email_tokens)
    
    #retorna a lista de emails tokenizados
    return(lista)

def email_vectorize(lista_emails, palavras_spam):
    
    sz1 = len(lista_emails)
    sz2 = len(palavras_spam)
    data = numpy.zeros((sz1, sz2), int)

    for i, email in enumerate(lista_emails):
        vector = numpy.zeros(sz2) 
        for j, item in enumerate(palavras_spam):
            if item in email:
                data[i][j] = 1
    
    return(data)

def listar_palavras_spam(spam_lista, size):
    
    palavras_spam = []    
    tudo = [item for sublist in spam_lista for item in sublist]
    for element in nltk.FreqDist(tudo).most_common(size):
        palavras_spam.append(element[0])
    
    return(palavras_spam)

In [ ]:
directory = ("/Users/andrevrpires/Downloads/ML/DadosSpam/spam/")
spam_lista = tokenize_directory(directory)

directory = ("/Users/andrevrpires/Downloads/ML/DadosSpam/nospameasy/")
nospam_lista = tokenize_directory(directory)

directory = ("/Users/andrevrpires/Downloads/ML/DadosSpam/hardham/")
hardnospam_lista = tokenize_directory(directory)

In [439]:
palavras_spam = listar_palavras_spam(spam_lista, 1000)

data_spam = email_vectorize(spam_lista, palavras_spam)
data_nospam = email_vectorize(nospam_lista, palavras_spam)
data_hardnospam = email_vectorize(hardnospam_lista, palavras_spam)

X = numpy.concatenate((data_spam, data_nospam, data_hardnospam), axis = 0)
y = numpy.concatenate((numpy.zeros(data_spam.shape[0], int),
                       numpy.ones(data_nospam.shape[0], int),
                       numpy.ones(data_hardnospam.shape[0], int)), axis = 0)

X, y = shuffle(X, y, random_state = 123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [440]:
model1 = LogisticRegression(solver = 'lbfgs').fit(X_train, y_train)
model1.score(X_test, y_test)

0.9924242424242424

In [441]:
model2 = svm.LinearSVC(max_iter = 3000).fit(X_train, y_train)
model2.score(X_test, y_test)

0.9919191919191919

In [442]:
model3 = svm.SVC(gamma='scale').fit(X_train, y_train)
model3.score(X_test, y_test)

0.9904040404040404